In [1]:
# import packages for transformations
import sqlite3
from cmath import nan

import pandas as pd
from pathlib import Path


In [ ]:
# Tables changed

## Tables changed in this script
1. vin_clinicaltrials

### Set connection to Database

In [2]:
#1. path to this script
current_dir = Path.cwd()

#2. Path to project root
project_root = current_dir.parent

#3. Go to database folder
db_path = project_root / "database" / "dataverse_complete.db"

#4. define the connection to the database
try:
    conn = sqlite3.connect(str(db_path))
    print("✅ Connection Successful!")
    print(f"Connected to: {db_path}")
except Exception as e:
    print(f"❌ Still failing. Looking at: {db_path}")
    print(f"Error: {e}")



✅ Connection Successful!
Connected to: /Users/lottesavelberg/Documents/Akvo/IGH/igh-data-transform/src/igh_data_transform/database/dataverse_complete.db


In [3]:
#if the connection was succesful, open it again
conn = sqlite3.connect(str(db_path))

In [4]:
#1. define the candidates table
table_name = "vin_clinicaltrials"
df = pd.read_sql_query(f"SELECT * FROM {table_name}", conn)

# see the first rows
print(f"Table' {table_name} was successfully loaded!")
df.head()

Table' vin_clinicaltrials was successfully loaded!


,row_id,new_studydesign,new_collaborator,new_studydocuments,new_test,vin_ctphase,vin_ctid,new_sponsor,new_fundertype,new_aim1ctlastupdated,...,new_secondaryoutcomemeasures,_owninguser_value,vin_ctstatus,utcconversiontimezonecode,statuscode,_owningteam_value,json_response,sync_time,valid_from,valid_to
0,1,None,None,None,None,1,4506,ViiV Healthcare,None,None,...,None,38c55be4-83e8-ed11-8848-00224818a395,909670002.0,None,1,None,"{""_createdby_value"": ""38c55be4-83e8-ed11-8848-...",2026-01-09T12:02:02.372914+00:00,2025-01-13T14:35:57Z,None
1,2,None,None,None,None,Unknown,4498,NIAID,None,None,...,None,38c55be4-83e8-ed11-8848-00224818a395,100000002.0,None,1,None,"{""_createdby_value"": ""38c55be4-83e8-ed11-8848-...",2026-01-09T12:02:02.412715+00:00,2024-12-19T10:08:40Z,None
2,3,None,Richmond Pharmacology Limited\nPharmaKinetic Ltd,None,None,I,4500,Medicines for Malaria Venture,None,None,...,None,38c55be4-83e8-ed11-8848-00224818a395,909670002.0,None,1,None,"{""_createdby_value"": ""38c55be4-83e8-ed11-8848-...",2026-01-09T12:02:02.422311+00:00,2025-01-06T13:19:15Z,None
3,4,None,None,None,None,I,4501,None,None,None,...,None,38c55be4-83e8-ed11-8848-00224818a395,100000004.0,None,1,None,"{""_createdby_value"": ""38c55be4-83e8-ed11-8848-...",2026-01-09T12:02:02.427973+00:00,2025-01-07T08:04:34Z,None
4,5,None,None,None,Phase IIINCT04897516,Phase III,4504,None,None,None,...,None,ea738897-37ae-eb11-8236-00224814fc41,100000002.0,None,1,None,"{""_createdby_value"": ""ea738897-37ae-eb11-8236-...",2026-01-09T12:02:02.437200+00:00,2025-01-13T10:08:07Z,None


In [5]:
#1. Calculate stats of each column
stats = []
for col in df.columns:
   stats.append({
       'column_name': col,
       'unique_values': df[col].nunique(),
       'empty_cells': df[col].isnull().sum(),
       'data_type': df[col].dtypes
   })

#2. Create data frame for the stats
df_stats = pd.DataFrame(stats)

#3. Sort stats data frame by number of unique values to identify index column
df_stats = df_stats.sort_values(by='unique_values', ascending=False)

#4. Print the results
print("Column Quality Summary:")
display(df_stats)

Column Quality Summary:


,column_name,unique_values,empty_cells,data_type
0,row_id,4794,0,int64
69,sync_time,4794,0,object
13,vin_clinicaltrialid,4794,0,object
68,json_response,4794,0,object
54,versionnumber,4794,0,int64
...,...,...,...,...
65,utcconversiontimezonecode,0,4794,object
67,_owningteam_value,0,4794,object
40,_modifiedonbehalfby_value,0,4794,object
30,overriddencreatedon,0,4794,object


In [7]:
#drop all columns with only empty cells
#1. Identify all column with only empty cells
empty_cols = df.columns[df.isnull().all()].tolist()

#2. Print empty colls
if not empty_cols:
    print("No columns are empty")
else:
    print(f"There are {len(empty_cols)} empty columns")
    for col in empty_cols:
        print(f" - {col}")

# Create a new list excluding 'valid_to'
cols_to_drop = [c for c in empty_cols if c != 'valid_to']

# Drop the filtered list
df_transformed = df.drop(columns=cols_to_drop)

There are 6 empty columns
 - _createdonbehalfby_value
 - overriddencreatedon
 - _modifiedonbehalfby_value
 - utcconversiontimezonecode
 - _owningteam_value
 - valid_to


In [8]:
def inspect_column(df, column_name):
    """
    Prints stats, missing data percentage, and top 10 values for a specific column.
    """
    # 1. Check if column exists
    if column_name not in df.columns:
        print(f"❌ Column '{column_name}' not found.")
        return

    print(f"--- Statistics for: {column_name} ---")

    # 2. Basic Stats & Data Type
    unique_vals = df[column_name].nunique()
    empty_cells = df[column_name].isnull().sum()
    filled_cells = df[column_name].count()
    dtype = df[column_name].dtype

    print(f"Unique values: {unique_vals} | Data type: {dtype} | filled cells: {filled_cells}" )

    # 3. Percentage Missing Data
    null_pct = (empty_cells / len(df)) * 100
    print(f"Empty cells: {empty_cells} ({null_pct:.2f}%)")

    #4. Show unique values if they are low (Categorical Data)
    if unique_vals < 20:
        print(f"\n--- All unique values (<20) ---")
        unique_list = df[column_name].dropna().unique()
        print(sorted(unique_list))
    else:
        print(f"\n--- First 10 values ---")
        print(df[column_name].value_counts().head(10))

    print("-" * 40 + "\n")

### Inspect columns and determine what to do with them

In [12]:
 df_transformed.columns.tolist()

['row_id',
 'new_studydesign',
 'new_collaborator',
 'new_studydocuments',
 'new_test',
 'vin_ctphase',
 'vin_ctid',
 'new_sponsor',
 'new_fundertype',
 'new_aim1ctlastupdated',
 'modifiedon',
 'importsequencenumber',
 'vin_ctterminatedreason',
 'vin_clinicaltrialid',
 'new_locations',
 'statecode',
 '_vin_candidate_value',
 'new_firstposted',
 'createdon',
 'new_aim1ctnumber',
 'new_outcomemeasure_secondary',
 'vin_ctresultstype',
 'vin_title',
 'vin_enddate',
 'new_includedaim1',
 '_ownerid_value',
 'new_aim1listsctid',
 '_modifiedby_value',
 'new_outcomemeasure_primary',
 'new_aim1ctstatus',
 'new_interventions',
 'vin_ctresultsstatus',
 'vin_ctenrolment',
 'vin_endtype',
 'new_aim1pcrreviewnotes',
 'new_age',
 'vin_startdate',
 'new_sex',
 'new_pipsct',
 '_createdby_value',
 'vin_ctrialid',
 'new_conditions',
 'vin_starttype',
 'vin_pcrreviewcomments',
 '_owningbusinessunit_value',
 'vin_description',
 'new_indicationtype',
 'vin_ctterminatedtype',
 'vin_ctresultssource',
 'vin_sou

In [105]:
 inspect_column(df_transformed, 'statuscode')


--- Statistics for: statuscode ---
Unique values: 2 | Data type: int64 | filled cells: 4794
Empty cells: 0 (0.00%)

--- All unique values (<20) ---
[np.int64(1), np.int64(2)]
----------------------------------------



In [57]:
# Remove all leading/trailing whitespace
df['new_age'] = df['new_age'].str.strip()

# Now check your counts again
#print(df['new_age'].value_counts())

# Get unique values as a list to see trailing/leading spaces
#print(df['new_age'].unique().tolist())

#### Columns to transform within dataframe

In [23]:
# Update CT phase in categorical values
def synthesize_phase(val):
    if pd.isna(val) or val == 'None':
        return 'Unknown'

    # Standardize for comparison
    v = str(val).upper().strip().replace('\n', ' ')

    # 1. N/A & Unknown (Specific non-phase labels)
    na_terms = ['N/A', 'NOT APPLICABLE', 'PHASE N/A', 'PHASE: N/A', 'NA', 'NOT SELECTED', '0', 'PHASE 0']
    if v in na_terms: return 'N/A'

    unknown_terms = ['UNKNOWN', 'PHASE UNSPECIFIED', 'OTHER', 'NOT SELECTED']
    if any(term in v for term in unknown_terms): return 'Unknown'

    # 2. Combined / Bridge Phases (Check these FIRST)
    if any(x in v for x in ['I/II', '1/2', '1 AND 2', '12', 'PHASE1|PHASE2']): return 'Phase I/II'
    if any(x in v for x in ['II/III', '2/3', 'PHASE2|PHASE3']): return 'Phase II/III'
    if any(x in v for x in ['III/IV', '3/4']): return 'Phase III/IV'

    # 3. Phase IV
    if any(x in v for x in ['PHASE 4', 'PHASE IV', 'IV', '4', 'POST-MARKET', 'POST MARKETING']):
        if not any(x in v for x in ['III', 'II', 'I']): return 'Phase IV'
        if 'PHASE IV): YES' in v: return 'Phase IV'

    # 4. Phase III (including IIIa/IIIb)
    if any(x in v for x in ['PHASE 3', 'PHASE III', 'III', '3', 'PHASE3']):
        if not any(x in v for x in ['IV']): return 'Phase III'

    # 5. Phase II (including IIa/IIb)
    if any(x in v for x in ['PHASE 2', 'PHASE II', 'II', '2', 'PHASR II', 'PHASE2']):
        if not any(x in v for x in ['III']): return 'Phase II'

    # 6. Phase I (including Ia/Ib)
    if any(x in v for x in ['PHASE 1', 'PHASE I', 'I', '1', 'EARLY_PHASE1', 'PHASE1']):
        return 'Phase I'

    # 7. Specialized Study Types (Preserve the original flavor)
    if 'OBSERVATIONAL' in v: return 'Observational'
    if 'INTERVENTIONAL' in v: return 'Interventional'
    if 'RETROSPECTIVE' in v: return 'Retrospective'
    if 'CHIM' in v: return 'CHIM'

    return 'Unknown'

# Apply the function
df_transformed['vin_ctphase'] = df_transformed['vin_ctphase'].apply(synthesize_phase)
print(df_transformed['vin_ctphase'].value_counts())

vin_ctphase
Unknown         1683
N/A              967
Phase I          878
Phase III        551
Phase II         498
Phase I/II       172
Phase IV          26
Phase III/IV      11
Phase II/III       8
Name: count, dtype: int64


In [59]:
def synthesize_gender(val):
    if pd.isna(val) or val == 'None':
        return 'Unknown'

    # Clean up artifacts and case
    v = str(val).upper().replace('<BR>', ' ').strip()

    # 1. Both (Check this first to catch 'Male: yes Female: yes' or 'Both')
    if any(x in v for x in ['BOTH', 'ALL', 'AND FEMALE', 'AND MALE']):
        return 'Both'

    # Logic for cases like 'Female: yes Male: no'
    if 'FEMALE: YES' in v and 'MALE: YES' in v:
        return 'Both'

    # 2. Female Only
    if v == 'F' or v == 'FEMALE' or v == 'FEMALES':
        return 'Female'
    if 'FEMALE: YES' in v and 'MALE: NO' in v:
        return 'Female'

    # 3. Male Only
    if v == 'M' or v == 'MALE' or v == 'MALES':
        return 'Male'
    if 'MALE: YES' in v and 'FEMALE: NO' in v:
        return 'Male'

    return 'Unknown'

# Apply the function
df_transformed['new_sex'] = df['new_sex'].apply(synthesize_gender)

In [82]:
def clean_study_types(val):
    if pd.isna(val):
        return val

    # Standardize for comparison (remove whitespace and case)
    v_clean = str(val).strip().upper()

    # 1. Handle Interventional variants
    interventional_variants = [
        'INTERVENTIONAL', 'INTERVENTIONAL STUDY',
        'INTERVENTION', 'INTERVENTIONAL CLINICAL TRIAL OF MEDICINAL PRODUCT'
    ]
    if v_clean in interventional_variants:
        return 'Interventional'

    # 2. Handle Observational variants
    observational_variants = [
        'OBSERVATIONAL', 'OBSERVATIONAL STUDY',
        'OBSERVATIONAL NON INVASIVE'
    ]
    if v_clean in observational_variants:
        return 'Observational'

    # 3. Return everything else exactly as it is (including "Basic science", "PMS", etc.)
    return val

# Apply to your column
df_transformed['new_studytype'] = df_transformed['new_studytype'].apply(clean_study_types)

# Verify the results
print(df_transformed['new_studytype'].unique().tolist())

[None, 'Interventional', 'Observational', 'Basic science', 'Observational [Patient Registry]', 'Relative factors research', 'Diagnostic test', 'Basic Science', 'Expanded Access:Individual Patients', 'PMS', 'Screening', 'Randomized, Parallel Group Trial<br>  Method of generating randomization sequence:Random Number Table  Method of allocation concealment:Sequentially numbered, sealed, opaque envelopes  Blinding and masking:Outcome Assessor Blinded', 'Prospective longitudinal observational study  (Diagnostic)', 'Other\n  Method of generating randomization sequence:Not Applicable  Method of allocation concealment:Not Applicable  Blinding and masking:Not Applicable', 'Observational Model: Case-Only, Time Perspective: Retrospective', 'N/A: single arm study, Open (masking not used), N/A , unknown, Parallel']


#### Columns to change in options set

In [89]:
#replace values in option set
#1. define the candidates table
table_name = "_optionset_vin_ctstatus"
df = pd.read_sql_query(f"SELECT * FROM {table_name}", conn)

# see the first rows
print(f"Table' {table_name} was successfully loaded!")
df.head(20)

Table' _optionset_vin_ctstatus was successfully loaded!


,code,label,first_seen
0,100000001,Planned,2026-01-09T12:02:04.983857+00:00
1,100000002,Recruiting,2026-01-09T12:02:02.418792+00:00
2,100000003,Not yet recruiting,2026-01-09T12:02:02.454898+00:00
3,100000004,"Active, not recruiting",2026-01-09T12:02:02.432984+00:00
4,100000005,Enrolling by invitation,2026-01-09T12:02:15.257026+00:00
5,100000006,Not Recruiting,2026-01-09T12:02:03.282247+00:00
6,909670000,Terminated,2026-01-09T12:02:04.391299+00:00
7,909670001,Active,2026-01-09T12:02:02.481115+00:00
8,909670002,Completed,2026-01-09T12:02:02.409603+00:00
9,909670003,Results submitted,2026-01-09T12:02:19.077735+00:00


In [90]:
#drop row 4
df_removed = df.drop([0, 1, 2,3,4,5, 9])

In [91]:
# save updated data as the optionset
# discuss with zudhil and jacob how to code this

In [103]:
# transform catagories in ct status
df_transformed['vin_ctstatus'] = df_transformed['vin_ctstatus'].replace({
  100000001.0: 909670001,
  100000002.0 : 909670001,
    100000003.0 : 909670001,
100000004.0 : 909670001,
100000005.0 : 909670001,
100000006.0 : 909670001,
909670003.0 :909670001

})


In [ ]:
def synthesize_status(val):
    if pd.isna(val):
        return 'Unknown'

    # Standardize for matching
    v = str(val).strip().upper()

    # 1. Active Category (The "Planned/Recruiting" group)
    active_terms = [
        'PLANNED', 'RECRUITING', 'NOT YET RECRUITING',
        'ACTIVE, NOT RECRUITING', 'ACTIVE - NOT RECRUITING',
        'ENROLLING BY INVITATION', 'NOT RECRUITING', 'ACTIVE'
    ]
    if any(term == v for term in active_terms):
        return 'Active'

    # 2. Completed (Including those with results submitted)
    if v in ['COMPLETED', 'RESULTS SUBMITTED']:
        return 'Completed'

    # 3. Direct Mappings (Preserving specific stop statuses)
    if 'TERMINATED' in v: return 'Terminated'
    if 'SUSPENDED' in v: return 'Suspended'
    if 'WITHDRAWN' in v: return 'Withdrawn'

    # 4. Default
    return 'Unknown'

# Apply the logic
df['study_status_clean'] = df['study_status_column'].apply(synthesize_status)

#### Columns to remove

In [106]:
### columns to remove
cols_to_drop = ['new_aim1ctlastupdated', 'new_aim1ctnumber', 'new_includedaim1', '_ownerid_value', 'new_aim1listsctid', '_modifiedby_value', 'new_aim1ctstatus',
                'new_aim1pcrreviewnotes', 'new_pipsct', '_createdby_value', 'vin_pcrreviewcomments', '_owningbusinessunit_value', 'new_resultsfirstposted',
                'versionnumber', 'new_primarycompletiondate', 'new_primaryoutcomemeasures', 'timezoneruleversionnumber', 'vin_lastupdated', 'new_secondaryoutcomemeasures', '_owninguser_value', 'json_response', 'sync_time']

#### Columns to change name

In [107]:
# update column name
df_transformed = df_transformed.rename(columns={"new_studydesign": "study_design",
                                                'new_collaborator': 'collaborator',
                                                'new_studydocuments': 'studydocuments',
                                                'new_test' : 'test',
                                                'vin_ctphase': 'ctphase',
                                                'vin_ctid': 'ctid',
                                                'new_sponsor':'sponsor',
                                                'new_fundertype': 'fundertype',
                                                'vin_ctterminatedreason': 'ctterminatedreason',
                                                'vin_clinicaltrialid': 'clinicaltrialid',
                                                'new_locations': 'locations',
                                                '_vin_candidate_value': 'candidate_value',
                                                'new_firstposted': 'firstposted',
                                                'new_outcomemeasure_secondary': 'outcomemeasure_secondary',
                                                'vin_ctresultstype': 'ctresultstype',
                                                'vin_title': 'title',
                                                'vin_enddate': 'enddate',
                                                'new_outcomemeasure_primary': 'outcomemeasure_primary',
                                                    'new_interventions': 'interventions',
                                                'vin_ctresultsstatus': 'ctresultsstatus',
                                                'vin_ctenrolment': 'cttenrolment',
                                                'vin_endtype': 'endtype',
                                                'new_age': 'age',
                                                'vin_startdate': 'startdate',
                                                'new_sex': 'sex',
                                                'vin_ctrialid': 'trialid',
                                                'new_conditions': 'conditions',
                                                'vin_starttype': 'starttype',
                                                'vin_description': 'description',
                                                'new_indicationtype': 'indicationtype',
                                                'vin_ctterminatedtype': 'ctterminatedtype',
                                                'vin_ctresultssource': 'ctresultssource',
                                                'vin_recentupdates': 'recentupdates',
                                                'vin_name': 'name',
                                                'new_studytype': 'studytype',
                                                'vin_ctstatus': 'ctstatus'})